In [1]:
import Plots
import StatsPlots
using Gen, Statistics, Memoize, Luxor


In [2]:
CONTROL_FACTOR = 50
AMOUNT_AGENT = 1000

function transition(state,action)
    if action == :down
        return [state[1]+1,state[2]]
    elseif action == :up
        return [state[1]-1, state[2]]
    elseif action == :left
        return [state[1], state[2]-1]
    elseif action == :right
        return [state[1], state[2]+1]
    end
end

function get_tile(state)
    return grid_world[state...]
end

function state_to_actions(state)
    actions = []
    if state[1] < grid_size[1] && get_tile(transition(state,:down)) != "#" 
        push!(actions,:down)
    end
    if state[1] > 1 && get_tile(transition(state,:up)) != "#"
        push!(actions,:up)
    end 
    if state[2] > 1 && get_tile(transition(state,:left)) != "#"
        push!(actions, :left)
    end 
    if state[2] < grid_size[2] && get_tile(transition(state,:right)) != "#"
        push!(actions,:right)
    end
    return actions
end

function is_terminal_state(state)
    tile = get_tile(state)
    return tile == "V" || tile == "DN" || tile == "DS" || tile == "N"
end

function sample(prob_dict)
    values = collect(keys(prob_dict))
    probs = collect(values(prob_dict))
    return values[rand(categorical(probs))]
end

function make_agent(utility_table)
    function utility(state)
        return utility_table[get_tile(state)]
    end

    @gen function act(state, time_left)
        possible_actions = state_to_actions(state)
        action_index = @trace(uniform_discrete(1,length(possible_actions)),:action_index)
        next_action = possible_actions[action_index]
        eu = expected_utility(state, next_action, time_left)
        @trace(bernoulli(exp(CONTROL_FACTOR * eu)), :factor)
        return action_index
    end

    @memoize Dict function run_act(state, time_left)
        action_indices = []
        trace, = generate(act, (state, time_left), choicemap((:factor,1)))
        for i = 1:AMOUNT_AGENT
            trace, = Gen.mh(trace, select(:action_index))
            push!(action_indices, get_retval(trace))
        end
        return action_indices
    end

    @gen function reward(state, action, time_left)
        next_state = transition(state, action)
        action_indices = run_act(next_state, time_left)
        rand_choice = @trace(uniform_discrete(1, length(action_indices)), :rand_choice)
        possible_actions = state_to_actions(next_state)
        next_action_idx = action_indices[rand_choice]
        next_action = possible_actions[next_action_idx]
        return expected_utility(next_state, next_action, time_left)
    end

    @memoize Dict function run_reward(state, action, time_left)
        rewards = []
        trace, = generate(reward, (state, action, time_left))
        for i =1:AMOUNT_AGENT
            trace, = Gen.mh(trace, select(:rand_choice))
            push!(rewards, get_retval(trace))
        end
        return rewards
    end

    @memoize Dict function expected_utility(state, action, time_left)
        u = utility(state)
        new_time_left = time_left - 1
        if is_terminal_state(state) || new_time_left == 0
            return u
        else
            return u + mean(run_reward(state, action, new_time_left))
        end
    end
    return run_act
end

@memoize Dict function simulate_agent(utility_table, start_state, total_time, total_steps=total_time)
    trajectory = []
    run_act = make_agent(utility_table)
    next_state = start_state
    while !is_terminal_state(next_state) && total_time > 0 && total_steps > 0
        action_indices = run_act(next_state, total_time)
        possible_actions =  state_to_actions(next_state)
        rand_choice = uniform_discrete(1, length(action_indices))
        next_action_idx = action_indices[rand_choice]
        next_action = possible_actions[next_action_idx]
        push!(trajectory,[next_state,next_action])
        next_state = transition(next_state, next_action)

        total_time -= 1
        total_steps -=1
    end
    if total_time == 0 || is_terminal_state(next_state)
        push!(trajectory, [next_state,:terminate])
    end  
    return trajectory
end;


(process:18736): GLib-GIO-WARNING **: 15:21:30.419: Unexpectedly, UWP app `XBMCFoundation.Kodi_19.0.500.0_x64__4n2hpmxwrvr6p' (AUMId `XBMCFoundation.Kodi_4n2hpmxwrvr6p!Kodi') supports 120 extensions but has no verbs
